In [5]:
# Import necessary libraries
import os
import cv2
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from skimage.metrics import structural_similarity as ssim



# Check if GPU is available

In [6]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


# Define data augmentation transforms

In [7]:

data_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
])

 # Define custom dataset class for dehazing

In [8]:

class DehazeDataset(Dataset):
    def __init__(self, img_dir, transform=None):
        self.img_dir = img_dir
        self.transform = transform
        self.img_list = os.listdir(img_dir)

    def __len__(self):
        return len(self.img_list)

    def __getitem__(self, idx):
        img_name = self.img_list[idx]
        img_path = os.path.join(self.img_dir, img_name)
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            image = self.transform(image)
        return image


# Define encoder-decoder architecture

In [9]:

class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

    def forward(self, x):
        x = self.encoder(x)
        return x

class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.decoder = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.Upsample(scale_factor=2, mode='nearest'),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 3, kernel_size=3, stride=1, padding=1),
        )

    def forward(self, x):
        x = self.decoder(x)
        return x


# Define model, loss function, optimizer

In [11]:
encoder = Encoder().to(device)
decoder = Decoder().to(device)
model = nn.Sequential(encoder, decoder)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)

# Define dataset and dataloader

In [10]:
gt_dir = '../minor_project/input/GT'
hazy_dir = '../minor_project/input/hazy'
gt_dataset = DehazeDataset(gt_dir, transform=data_transforms)
hazy_dataset = DehazeDataset(hazy_dir, transform=data_transforms)
gt_dataloader = DataLoader(gt_dataset, batch_size=1, shuffle=True)
hazy_dataloader = DataLoader(hazy_dataset, batch_size=1, shuffle=True)


# Training loop

In [13]:

num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for gt_img, hazy_img in zip(gt_dataloader, hazy_dataloader):
        gt_img = gt_img.to(device)
        hazy_img = hazy_img.to(device)

        optimizer.zero_grad()

        # Forward pass
        output = model(hazy_img)

        # Compute loss
        loss = criterion(output, gt_img)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(gt_dataloader)}")

OutOfMemoryError: CUDA out of memory. Tried to allocate 3.15 GiB. GPU 0 has a total capacity of 21.96 GiB of which 1.52 GiB is free. Process 4806 has 972.00 MiB memory in use. Including non-PyTorch memory, this process has 19.48 GiB memory in use. Of the allocated memory 19.21 GiB is allocated by PyTorch, and 47.63 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:

# Evaluate PSNR and SSIM
def compute_metrics(gt_images, dehazed_images):
    psnr_sum = 0.0
    ssim_sum = 0.0
    for gt_img, dehazed_img in zip(gt_images, dehazed_images):
        psnr_sum += cv2.PSNR(gt_img, dehazed_img)
        ssim_sum += ssim(gt_img, dehazed_img, multichannel=True)
    mean_psnr = psnr_sum / len(gt_images)
    mean_ssim = ssim_sum / len(gt_images)
    return mean_psnr, mean_ssim

In [ ]:


# Apply model to hazy images and compute metrics
dehazed_images = []
with torch.no_grad():
    for hazy_img in hazy_dataloader:
        hazy_img = hazy_img.to(device)
        dehazed_img = model(hazy_img)
        dehazed_images.append(dehazed_img.cpu().numpy())

mean_psnr, mean_ssim = compute_metrics(gt_dataset, dehazed_images)
print(f"Mean PSNR: {mean_psnr}, Mean SSIM: {mean_ssim}")
